In [1]:
import pandas as pd

df_customers = pd.read_csv('customer_details.csv')
df_customers.head()

,customer_id,sex,customer_age,tenure
0,9798859,Male,44.0,93
1,11413563,Male,36.0,65
2,818195,Male,35.0,129
3,12049009,Male,33.0,58
4,10083045,Male,42.0,88


Conhecendo melhor os dados

In [2]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   customer_id   20000 non-null  int64  
 1   sex           20000 non-null  object 
 2   customer_age  20000 non-null  float64
 3   tenure        20000 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 625.1+ KB


Buscando valores NaN(Not a Number) no dataframe:

In [3]:
df_customers.isna().sum()

customer_id     0
sex             0
customer_age    0
tenure          0
dtype: int64

Nota-se valores fora do comum na coluna 'sex'

In [4]:
df_customers.sex.value_counts()

sex
Male                15322
Female               4669
kvkktalepsilindi        8
UNKNOWN                 1
Name: count, dtype: int64

Foi decidido trocar os valores incoerentes por 'Female', já que ela se encontra em menor nº no Df

In [5]:
df_customers['sex'] = df_customers['sex'].replace(['kvkktalepsilindi', 'UNKNOWN'], 'Female')
df_customers.sex.value_counts()

sex
Male      15322
Female     4678
Name: count, dtype: int64

Pegando os valores maximos e minimos da idade dos clientes

In [6]:
df_customers.customer_age.max(), df_customers.customer_age.min()

(2022.0, -34.0)

Foi visto que há diversos valores que não se coincidem com o estilo de vida humano, assim decidimos muda-los

In [7]:
normal_age = df_customers.loc[(df_customers.customer_age < 110) & (df_customers.customer_age >= 16)]
normal_age.sort_values('customer_age',ascending = False)

,customer_id,sex,customer_age,tenure
12354,18249203,Male,103.0,27
7204,32741149,Female,103.0,10
7856,30022394,Male,101.0,12
8140,28411257,Male,101.0,14
8622,27629553,Female,100.0,15
...,...,...,...,...
2561,11084465,Male,16.0,70
10186,23195039,Female,16.0,19
19288,13073879,Male,16.0,48
7728,30420705,Male,16.0,12


Com isso, vamos pegar a idade media de cada sexo e fazer a troca dos valores errados

In [8]:
age_mean = normal_age.groupby('sex')['customer_age'].mean()
age_mean

sex
Female    35.082235
Male      33.358871
Name: customer_age, dtype: float64

In [9]:
# Substituição feita
new_old = (df_customers.customer_age > 110) | (df_customers.customer_age < 16)
df_customers.loc[new_old & (df_customers['sex']=='Male'), 'customer_age'] = 33
df_customers.loc[new_old & (df_customers['sex']=='Female'), 'customer_age'] = 35

df_customers.customer_age.max(),df_customers.customer_age.min() 

(103.0, 16.0)

FIM LIMPEZA E ANALISE CUSTOMERS / INICIO LIMPEZA E ANALISE BASKET

In [10]:
df_basket = pd.read_csv('basket_details.csv')
df_basket.head()

,customer_id,product_id,basket_date,basket_count
0,42366585,41475073,2019-06-19,2
1,35956841,43279538,2019-06-19,2
2,26139578,31715598,2019-06-19,3
3,3262253,47880260,2019-06-19,2
4,20056678,44747002,2019-06-19,2


In [11]:
df_basket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   15000 non-null  int64 
 1   product_id    15000 non-null  int64 
 2   basket_date   15000 non-null  object
 3   basket_count  15000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 468.9+ KB


Tranformar a coluna 'basket_date' em datetime Dtype para melhor uso

In [12]:
df_basket['basket_date'] = pd.to_datetime(df_basket['basket_date'], errors='coerce')
df_basket.dtypes

customer_id              int64
product_id               int64
basket_date     datetime64[ns]
basket_count             int64
dtype: object

Nova coluna para representar as compras no mes-ano, para ter uma melhor analise dos dados depois

In [13]:
df_basket['year_month'] = df_basket['basket_date'].dt.to_period('M').astype(str)

In [14]:
df_basket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   customer_id   15000 non-null  int64         
 1   product_id    15000 non-null  int64         
 2   basket_date   15000 non-null  datetime64[ns]
 3   basket_count  15000 non-null  int64         
 4   year_month    15000 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 586.1+ KB


In [15]:
df_basket.basket_count.max(),df_basket.basket_count.min()

(10, 2)

In [16]:
# Produtos comprados por mes
monthly_sales = df_basket.groupby('year_month')['basket_count'].sum()
monthly_sales

year_month
2019-05    19209
2019-06    13097
Name: basket_count, dtype: int64

In [17]:
# Total de compras feitas por clientes
customer_sales = df_basket.groupby('customer_id')['basket_count'].sum().sort_values(ascending = False)
customer_sales.head()

customer_id
8276934     29
7097820     18
11082470    17
11732494    14
22988999    14
Name: basket_count, dtype: int64

Salvando os arquivos limpos e prontos para analise

In [18]:
df_basket.to_csv('basket_clean.csv', index=False)
df_customers.to_csv('customer_clean.csv', index=False)